In [1]:
import time
print(time.ctime())
%load_ext ipydex.displaytools

Fri Feb 25 17:09:17 2022


This notebook contains an answer to a [question on stackoverflow](https://stackoverflow.com/questions/71266315/how-to-make-sympy-only-solve-the-real-results-when-solve-higher-order-equations?noredirect=1#comment125973419_71266315). The author would be glad about some feedback:

In [9]:
contact = 'ed:nedserd-ut llonk.netsrac'[::-1].replace(" ", "@")
# print(contact)

In [2]:
from sympy import *
from sympy.solvers.solveset import nonlinsolve

import sympy as sp

def clean_numbers(expr):
    
    numbers = expr.atoms(sp.Number)
    rplmts = [(n, sp.Rational(n)) for n in numbers]
    return expr.subs(rplmts)


h_0 = 170
b = 120
Es = 205
h = 200
M = 19760400
fc = 83
fy = 585


Ec_f, As_f, ft_f = (42.4, 226, 3)
vp_c, vp_s, x_n = xx = symbols('vp_c, vp_s, x_n', real=True)

f1 = vp_c * (h_0 - x_n) - x_n * vp_s # ← converted to a polynomial equation

f2 = 1 / 2 * Ec_f * vp_c * b * x_n - As_f * Es * vp_s - ft_f * b * (h - x_n)
f3 = M - As_f * Es * vp_s * (h_0 - x_n / 3) - ft_f * b * (h - x_n) * (h / 2 + x_n / 6)

ff = clean_numbers(Matrix([f1, f2, f3]).expand())

In [3]:
# reformulate the polynomial equations in terms of a groebner basis
# this set of equations has the same set of solutions but is described by different (often easier) equations
# note that this description depends on the (lexical) order of the variables
gb = groebner(ff, xx, order="lex")

# convert to matrix for easier access
ff2 = Matrix(gb.args[0]) ##:

ff2 := Matrix([
[vp_c - 1209393*x_n**3/26678185831300 - 47226832281*x_n**2/2262310158494240 - 208602595053*x_n/24590327809720 - 11012927625/24415175464],
[                     vp_s - 5724*x_n**3/392326262225 - 2208627*x_n**2/156930504890 - 20265354*x_n/3411532715 - 25600972662/15693050489],
[                                                x_n**4 + 192765*x_n**3/424 + 38473005*x_n**2/212 + 1132652675*x_n/106 - 67823066125/53]])

---

In [4]:
# solve the last equation (only depends on one variable)

sol2 = solve(ff2[-1], xx[-1]) ##

In [5]:
sol = []
for s in sol2:
    
    tmp_eqns = ff2[:2, :].subs(xx[-1], s.evalf())
    tmp_sol = solve(tmp_eqns, xx[:2]) # → dict
    tmp_sol[xx[-1]] =  s.evalf()
    sol.append(tmp_sol)
   

In [6]:
print(sol)

[{vp_c: 1.00656308634739, vp_s: 2.01548035330581, x_n: 56.6225231688573}, {vp_c: -0.469378305833599, vp_s: 1.02614290514871, x_n: -143.317861965125}]
